Preparing to submit wold stranded samples....

The spreasheet has a bunch of library accessions in the files accession column which is wrong. I think it needs to be fixed.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [4]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-22563-22581.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [5]:
award = 'UM1HG009443'

# Register Biosamples

In [6]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)


# Register Libraries

In [7]:

libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)


# Register Experiments

In [8]:

experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)


In [34]:
records = []
for i, experiment in experiments.iterrows():
    experiment_accession = experiment['accession']
    if experiment_accession.startswith('ENCSR'):
        obj = server.get_json(experiment_accession)
        for file_accession in obj['original_files']:
            f = server.get_json(file_accession)
            records.append({
                'md5sum': f['md5sum'],
                'uuid': f['uuid'],
                'file_accession': f['accession'],
                'submitted_file_name': f['submitted_file_name']
            })

In [37]:
submitted_files = pandas.DataFrame(records)

# Register Replicates

In [9]:

replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)


# Check Files

In [10]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)


In [39]:
fixed_files = pandas.merge(files, submitted_files, left_on='md5sum', right_on='md5sum')

In [40]:
fixed_files.to_excel('/dev/shm/fixed_files.xlsx')